In [ ]:
%load_ext line_profiler
%load_ext autoreload
%autoreload 2


In [ ]:
import os 
import sys
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
import osmnx as ox
%matplotlib qt

from strm2stl.buildings import *
from strm2stl.dem2stl import *
from strm2stl.osm2stl import *
import strm2stl.numpy2stl as np2stl

In [ ]:
#sys.path.append('..\\numpy2stl')



In [ ]:
import trimesh
import glob


In [ ]:
from strm2stl.create import *

In [ ]:
Root = "C:/Users/eac84/Desktop/Desktop/FILES"


In [ ]:
%matplotlib qt

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:

gdf = ox.features_from_place("Granada,Spain", tags)
gdf = ox.features_from_place("Albaicín,Spain", tags)

## Granada

In [ ]:

scale = .0003
center = (37.17827, -3.598) 
tags = {'building': True}

dist = 1500

models = location2solid(center, scale, tags, dist)


In [ ]:
print(vertices.mean(axis=0).round(-2))
print((vertices.max(axis=0) - vertices.min(axis=0)).round(2) )
print(vertices.max(axis=0).round(-2) , vertices.min(axis=0).round(-2))

print("")
print(vx.mean(axis=0).round(-2))
print( (vx.max(axis=0)- vx.min(axis=0)).round(2))
print(vx.max(axis=0).round(-2) ,  vx.min(axis=0).round())

In [ ]:
import napari
print("opening - Napari")
v = napari.current_viewer()
if v is None: v = napari.Viewer()
v.layers.clear()

for key in models:

	vertices, faces = models[key]
	surface = (vertices,faces)
	s = v.add_surface(surface)
	s.wireframe.visible = False

#s.normals.face.visible = True
#s.normals.vertex.visible = True

In [ ]:
for keys in models:

	vertices, faces = models[keys]
	surface = (vertices,faces)
	s = v.add_surface(surface)
	s.wireframe.visible = False

	mesh = trimesh.Trimesh(vertices,faces)
	mesh.export("Granada_" +keys+".stl")


In [ ]:
vertices2 = []
faces2 = []
maxfaces = 0

for keys in models:

	vertices, faces = models[keys]

	faces = faces + maxfaces
	print(faces.max())
	print(len(vertices))
	vertices2.append(vertices)
	faces2.append(faces)

	maxfaces = maxfaces + (faces.max()+1)

vertices2 = np.concatenate(vertices2,axis=0)
faces2    = np.concatenate(faces2, axis=0)

In [ ]:
mesh = trimesh.Trimesh(vertices2,faces2)
mesh.export("Granada_together.stl")

In [ ]:
models

In [ ]:
from skimage import filters,transform
 
outshape = np.array(data.shape)*2
filt = transform.resize(data, outshape)
filt = filters.gaussian( filt , sigma=3, truncate=3)
filt = filters.median(filt, np.ones((3,3)))

plt.figure()
plt.imshow(filt)

#data = dem.data
facet = np2stl.numpy2stl(filt)
solid = np2stl.Solid(facet)
#solid.simplify()
fn = "granada_topo.stl" 
solid.save_stl(fn)

In [ ]:
x = 660
y = 1470
plt.imshow(im[-780:-570,1370:1570])

In [ ]:
r0 = 2820
r1 = 3030
c0 = 1370
c1 = 1570

mx = im[r0:r1, c0:c1]
plt.imshow(mx)

In [ ]:
plt.imshow(im)
plt.plot(Wcoor,Ncoor, "ro")

In [ ]:
ax = plt.figure().add_subplot(projection='3d')
ax.plot(Ncoor,Wcoor,H, "o")

## Barranquilla

In [ ]:
#gdf["height"] = 30

gdf["heights"] = 30#building_heights(gdf)*.05
gdf["topo_base"] = 0
gdf["z0"] = gdf["topo_base"]
gdf["z1"] = gdf["z0"]+ gdf["heights"]

building_poly = get_polygons(gdf)
tris = triangulate_buildings(building_poly)

In [ ]:
vertices, faces = np2stl.vertices_to_index(tris)
vertices[:,[1,0]] = vertices[:,[0,1]]
vertices[:,2] = vertices[:,2] * scale
vertices = vertices*1000

In [1]:
import napari
print("opening - Napari")
v = napari.current_viewer()
if v is None: v = napari.Viewer()
v.layers.clear()

surface = (vertices,faces)
s = v.add_surface(surface)
s.wireframe.visible = False

opening - Napari


c:\Users\eac84\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\eac84\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\eac84\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


NameError: name 'vertices' is not defined

In [ ]:
import matplotlib.pyplot as plt

import osmnx as ox


In [ ]:

# Define the place or bounding box
n,s,e,w = (11.058,10.92,-74.75,-74.85)

G = ox.graph_from_bbox(north=n,south=s,east=e,west=w, network_type='all')
# Download the data for all roads
#G = ox.graph_from_place(place, network_type='all')


In [ ]:
dem = DEM(Root, (n,s,e,w))

In [ ]:
demdata = dem.data

In [ ]:
x = dem.data.copy()
x = ndi.morphological_gradient(dem.data, (3,3))<0.001
x = 1.*x + 1.*(dem.data.clip(0,3)<0.15)

In [ ]:
x = rescale(x,sz_out=2000) 
x = x/x.max()

demdata = rescale(demdata,sz_out=2000)

demdata = demdata / demdata.max() * 10 
demdata = demdata-(x*2.)

In [ ]:
# Plot the graph at different zoom levels
fig, ax = plt.subplots(1, 2, figsize=(20, 10))

# Zoomed-out view (low zoom level)
ox.plot_graph(ox.project_graph(G), ax=ax[0], node_size=0, edge_color='grey', edge_linewidth=0.2)
ax[0].set_title("Zoomed-Out View")

plt.show()

In [ ]:
def graph2lines(G):

	rivers = []
	for u, v, data in G.edges(keys=False, data=True):
		
		name = ""
		if "name" in data:  
			name = data["name"]         

		if "geometry" in data: 
			pts = np.array(data["geometry"].xy)
		else:
			node = G.nodes
			pts = [[ node[u]['x'], node[v]['x']],
				[ node[u]['y'], node[v]['y']]]
			pts = np.array(pts)

		line = Line(name, pts, color="g", type="river")
		rivers.append(line)
	return rivers

In [ ]:
rivers2 = graph2lines(G)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))

for line in rivers2:
    x,y = line.pts
    ax[0].plot(x,y,"g")
    
ax[0].grid(True)

In [ ]:
linepts = [line.pts for line in rivers2]

lineptsim = [coor2im(((n,s),(w,e)), ((0,demdata.shape[0]),(0,demdata.shape[1])), pts.T) 
             for pts in linepts]

lines_im = embed_lines(np.zeros_like(demdata).T, lineptsim , res=1).T

In [ ]:
building_im = morphology.area_opening(lines_im==0, 500) == 0
building_im = building_im & ~(lines_im)

In [ ]:
plt.imshow(building_im)

In [ ]:
landscape = demdata + building_im *5 + lines_im*-1 

In [ ]:
landscape = landscape - landscape.min() + 1

In [ ]:
savefile(out_dir, name, im2 )

In [ ]:
plt.imshow(landscape, cmap="jet")

In [3]:
facet = np2stl.numpy2stl(landscape)
solid = np2stl.Solid(facet)
vx = solid.vertices.copy().astype(float)
########################


NameError: name 'np2stl' is not defined

In [2]:
np2stl.numpy2stl

NameError: name 'np2stl' is not defined

In [ ]:
import napari
print("opening - Napari")
v = napari.current_viewer()
if v is None: v = napari.Viewer()
v.layers.clear()

surface = (vx,solid.faces)
s = v.add_surface(surface)
s.wireframe.visible = False

In [ ]:
import strm2stl.numpy2stl.numpy2stl as np3D

def savefile(out_dir, name, im2 ):
	if not os.path.isdir(out_dir): return

	out_dir = out_dir + name 
	os.makedirs(out_dir,exist_ok=True)
	
	tri = np3D.numpy2stl(im2)
	facets = np3D.triangles_to_facets(tri)
	print("Saving")
	np3D.writeSTL(facets, out_dir + "/" + name + ".stl")

In [ ]:
out_dir = "C:/Users/eac84/Desktop/Desktop/Tasks/STL/Maps/"

savefile(out_dir, "Barranquilla", landscape )

## SPA Race Track

In [ ]:
from osmnx import utils_geo

In [ ]:
#gdf = ox.features_from_place("Spa, Belgium", tags)

N,W = 50.438343, 5.970514
bbox = utils_geo.bbox_from_point((N,W), dist=2000)
n,s,e,w = bbox
dem = DEM(Root, (n,s,e,w))
G =  ox.graph_from_bbox( *bbox, network_type='all')
rivers2 = graph2lines(G)




In [ ]:
tags = {'building': True}
gdf = ox.features.features_from_bbox(*bbox, tags = tags)

In [ ]:
gdf

In [ ]:

fig, ax = plt.subplots(1, 1, figsize=(20, 10))

for line in rivers2:
    x,y = line.pts
    ax[0].plot(x,y,"g")
    
ax[0].grid(True)

In [ ]:
dem.draw()

In [ ]:

plt.figure()
for u, v, data in G.edges(keys=False, data=True):

	color = "g"     
	if "name" in data:  
		
		name = data["name"]
		color = "b"     
		print(name)
		if "Circuit" in name:
			color = "r"


	if 'highway' in data:
		if data["highway"] == "track":
			color = "m"
			continue

	if 'ref' in data: 
		color = "y"
		#'N62c'

	if "geometry" in data: 
		pts = np.array(data["geometry"].xy)
	else:
		node = G.nodes
		pts = [[ node[u]['x'], node[v]['x']],
			[ node[u]['y'], node[v]['y']]]
		pts = np.array(pts)

	plt.plot(pts[0],pts[1], color = color)			


In [92]:
polygons = []
for n,row in enumerate(gdf.itertuples(index=False)):
	geometry = row.geometry

	if isinstance(geometry, Polygon):
		pts = np.array(geometry.exterior.xy)
		poly = {"points":pts}
		polygons.append(poly)
	elif isinstance(geometry, MultiPolygon):
		#if geometry is multipolygon, go through each subpolygon 
		for subpolygon in geometry.geoms: 
			pts = np.array(subpolygon.exterior.xy)
			poly = {"points":pts}
			polygons.append(poly)


In [96]:
for p in polygons:
  x,y = p["points"]
  plt.fill(x,y)

In [97]:
plt.grid(True)

c:\Users\eac84\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from strm2stl.create import coor2im

In [ ]:
demdata = rescale(demdata,sz_out=1000)

In [ ]:
linepts = [line.pts for line in rivers2]

lineptsim = [coor2im(((n,s),(w,e)), ((0,demdata.shape[0]),(0,demdata.shape[1])), pts.T) 
             for pts in linepts]

lines_im = embed_lines(demdata.T, lineptsim , res=1).T

In [ ]:
lines_im = 50*embed_lines(demdata.T, lineptsim , res=1).T + demdata

In [ ]:
out_dir = "C:/Users/eac84/Desktop/Desktop/Tasks/STL/Maps/"

lines_im = lines_im - lines_im.min() + 1
savefile(out_dir, "Spa", lines_im )

In [ ]:
plt.figure()
plt.imshow(lines_im )

In [ ]:
demdata.min(), demdata.max()